<a href="https://colab.research.google.com/github/DGkurnia/discuss1/blob/main/airquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** [Derfansyah Guswiranata Kurnia]
- **Email:** [weerakurnia@gmail.com]
- **ID Dicoding:** [dgkurnia]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1 : Bagaimana kondisi suhu di dua belas kota dari tahun 2013 samapai 2017 ?
- Pertanyaan 2 : Bagaimana pemeriksaan ukuran partikuler (untuk PM 2.5 dan PM 10) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 3 : Bagaimana pemeriksaan kadar sulfur dioksida (SO2), kadar nitrogen monoksida (NO) dan karbon monoksida (CO) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 4 : Bagaimana pemeriksaan kondisi gas ozone (O3) dari tahun 2013 sampai 2017 ?.

In [2]:
from google.colab import drive
drive.mount('/content/drive') #dasar instalasi file dari Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Impor Semua Paket/Library yang Digunakan
Kumpulan perpustakaan Python yang dipakai untuk proyek:


In [3]:
#persiapan perpustakaan dan file untuk pembersihan
import numpy as np
import pandas as pd #dasar pengelolaan data
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime # dasar pengelolaan data berbasis waktu
from scipy.interpolate import interp1d #interpolasi linear
!pip install pykrige
from pykrige.ok import OrdinaryKriging as okrig #teknik k-riging

## Data Wrangling
Pada tahap ini, jumlah dataframe yang akan dianalisis adalah 12 unit

In [4]:

#persiapan insersi data
dfaotizhongxin = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/aotizhongxin.csv')
dfchangping = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/changping.csv')
dfdingling = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dingling.csv')
dfdongsi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dongsi.csv')
dfguanyuan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/guanyuan.csv')
dfgucheng = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/gucheng.csv')
dfhuairou = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/huairo.csv')
dfnonzhanguan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/nonzhanguan.csv')
dfshunyi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/shunyi.csv')
dftiantan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/tiantan.csv')
dfwanliu = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanliu.csv')
dfwanshouxigong = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanshouxigong.csv')
"""
Kota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong
"""

'\nKota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong\n'

### Tahap Pengumpulan (Gathering Data)
#### rincian dari pengumpulan data : ada 12 dataset kota yang terdiri dari
* Nomor
* Tahun
* Bulan
* Jam
* kadar partikulat (PM 2.5 dan 10)
* kadar senyawa SO2, NO, CO, dan O3 (dalam mikrogram/ m^3)
* tempratur (dalam celsius)
* Nilai tekanan (dalam hektoPascal)
* arah angin dan kecepatan angin
-- Nilai Tahun, bulan dan jam bisa diubah menjadi tanggal.


### Assessing Data
#### Hal yang diperhatikan dalam pemeriksaan kadar udara
- Total data duplikat
-- Aotizhongxin : tidak ada
-- Changpin : Tidak ada
- Jumlah data hilang di masing-masing unit
-- Kota Aotizhingxin : 925 data di PM2.5, 718 di PM 10. Tiga kolom memiliki data kosong di atas 500
-- Kota Changpin : banyak data hilang berada di pemeriksaan empat gas tersebut
--- Kota lain memiliki kecendrungan untuk memiliki data kosong di kondisi tersebut
#### Hal unik lain
Forrmat tanggal yang salah (awalnya tahun, bulan, hari, dan jam) yang dipisah

In [8]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota pertama :
(kota Aotizhongxin sampai Huairo)

#dfaotizhongxin.info()
#dfaotizhongxin.isnull().sum()
#dfaotizhongxin.duplicated().sum() # kota Aotizhongxin
#dfchangping.info()
#dfchangping.isnull().sum()
#dfchangping.duplicated().sum() # Changping
#dfdongsi.isnull().sum()
#dfdongsi.duplicated().sum()
#dfdingling.info()
#dfdingling.isnull().sum()
#dfdingling.duplicated().sum() #Dingling
"""

'\npemeriksaan data dari data kurang lengkap untuk enam kota pertama :\n(kota Aotizhongxin sampai Huairo)\n\n#dfaotizhongxin.info()\n#dfaotizhongxin.isnull().sum()\n#dfaotizhongxin.duplicated().sum() # kota Aotizhongxin\n#dfchangping.info()\n#dfchangping.isnull().sum()\n#dfchangping.duplicated().sum() # Changping\n#dfdongsi.isnull().sum()\n#dfdongsi.duplicated().sum()\n#dfdingling.info()\n#dfdingling.isnull().sum()\n#dfdingling.duplicated().sum() #Dingling\n'

In [7]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota terakhir :
(kota Nonzhanguan sampai Wanshouxigong)

#dfnonzhanguan.info()
#dfnonzhanguan.isnull().sum()
#dfnonzhanguan.duplicated().sum() #Nonzhanguan
#dfwanshouxigong.info()
"""

'\npemeriksaan data dari data kurang lengkap untuk enam kota terakhir :\n(kota Nonzhanguan sampai Wanshouxigong)\n\n#dfnonzhanguan.info()\n#dfnonzhanguan.isnull().sum()\n#dfnonzhanguan.duplicated().sum() #Nonzhanguan\n#dfwanshouxigong.info()\n'

### Cleaning Data
#### Pengantar : Data ini bisa dimanfaatkan untuk pengaturan sehingga proses yang sesuai untuk pembersihan data adalah interpolasi kolom indikator yang berangka demi menjaga aspek kolom yang hilang. Di sisilain, pengisian data dengan modus berlaku untuk kolom non integer
Tahapan pembersihan data
(hal ini berlaku untuk 12 kota)
1.   Pengubahan format waktu dan menghapus format salah (Tahun, bulan, angka, dan jam)
2. Proses interpolasi untuk kolom hilang di indikator
3. Pengisian nilai non numerik yang hilang




In [9]:
#pembersihan bagian pertama (pengubahan format data waktu) untuk dua belas kota
dfaotizhongxin['datetime'] = dfaotizhongxin[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfchangping['datetime'] = dfchangping[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdingling['datetime'] = dfdingling[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdongsi['datetime'] = dfdongsi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfguanyuan['datetime'] = dfguanyuan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfgucheng['datetime'] = dfgucheng[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfhuairou['datetime'] = dfhuairou[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfnonzhanguan['datetime'] = dfnonzhanguan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfshunyi['datetime'] = dfshunyi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dftiantan['datetime'] = dftiantan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanliu['datetime'] = dfwanliu[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanshouxigong['datetime'] = dfwanshouxigong[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)

#penghapusan kolom 'tahun' dan tiga indeks lain untuk 12 kota (dari Aotizhongxin sampai Wanshouxigong)

dfaotizhongxin.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfchangping.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdingling.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdongsi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfguanyuan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfgucheng.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfhuairou.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfnonzhanguan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfshunyi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dftiantan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanliu.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanshouxigong.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)

#pemeriksaan kolom untuk memeriksa jika kota sudah memiliki indikator waktu
#dfaotizhongxin.head(10) #Kota Aotizhongxin
#dfchangping.head(10)# Kota Changping
#dfdingling.head(10)# Kota Dingling
#dfdongsi.head(10)# Kota Dongsi
#dfguanyuan.head(10)# Kota Guanyuan
#dfgucheng.head(10)# Kota Gucheng
#dfhuairou.head(10)# Kota Huairou
#dfnonzhanguan.head(10) #kota Nanzhanguan
#dfshunyi.head(10) #kota sunyi
#dftiantan.head(10) #Kota Tiantan
#dfwanliu.head(10) #Kota Wanliu
#dfwanshouxigong.head(10) #Kota Wanshouxigong
#hasil ini untuk memeriksa penghapusan kolom waktu

In [10]:
"""
#bagian dua (proses interpolasi kolom)
Tahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong
"""
# pemeriksan total kolom kosong untuk interpolasi
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Aotizhongxing)
#dfchangping.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Changping)
#dfdingling.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dingling)
#dfdongsi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dongsi)
#dfguanyuan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Guanyuan)
#dfgucheng.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Gucheng)
#dfhuairou.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Huairou)
#dfnonzhanguan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Nonzhanguan)
#dfshunyi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Shunyi)
#dftiantan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Tiantan)
#dfwanliu.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanliu)
#dfwanshouxigong.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanshouxigong)

'\n#bagian dua (proses interpolasi kolom)\nTahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong\n'

##Resume Kolom Hilang Data untuk Interpolasi Linear:

*   Sebagian besar kolom hilang bisa 'diinterpolasi' untuk menjaga orisinilitas data
*   Proses ini melibatkan dua belas Kota



In [11]:
# persiapan interpolasi (Aotizhongxin sampai terakhir) di kolom PM2.5 dan PM10 sebelum pembersihan data
dfaotizhongxin['PM2.5'] = dfaotizhongxin['PM2.5'].interpolate(method='linear')
dfaotizhongxin['PM10'] = dfaotizhongxin['PM10'].interpolate(method='linear')
#Area Aotizhongxin
dfchangping['PM2.5'] = dfchangping['PM2.5'].interpolate(method='linear')
dfchangping['PM10'] = dfchangping['PM10'].interpolate(method='linear')
#Area Changping
dfdingling['PM2.5'] = dfdingling['PM2.5'].interpolate(method='linear')
dfdingling['PM10'] = dfdingling['PM10'].interpolate(method='linear')
#Area Dingling
dfdongsi['PM2.5'] = dfdongsi['PM2.5'].interpolate(method='linear')
dfdongsi['PM10'] = dfdongsi['PM10'].interpolate(method='linear')
#Area Dongsi
dfguanyuan['PM2.5'] = dfguanyuan['PM2.5'].interpolate(method='linear')
dfguanyuan['PM10'] = dfguanyuan['PM10'].interpolate(method='linear')
#Area Guanyuan
dfgucheng['PM2.5'] = dfgucheng['PM2.5'].interpolate(method='linear')
dfgucheng['PM10'] = dfgucheng['PM10'].interpolate(method='linear')
#Area Gucheng
dfhuairou['PM2.5'] = dfhuairou['PM2.5'].interpolate(method='linear')
dfhuairou['PM10'] = dfhuairou['PM10'].interpolate(method='linear')
#Area Huairou
dfnonzhanguan['PM2.5'] = dfnonzhanguan['PM2.5'].interpolate(method='linear')
dfnonzhanguan['PM10'] = dfnonzhanguan['PM10'].interpolate(method='linear')
#Area Nonzhanguan
dfshunyi['PM2.5'] = dfshunyi['PM2.5'].interpolate(method='linear')
dfshunyi['PM10'] = dfshunyi['PM10'].interpolate(method='linear')
#Area Shunyi
dftiantan['PM2.5'] = dftiantan['PM2.5'].interpolate(method='linear')
dftiantan['PM10'] = dftiantan['PM10'].interpolate(method='linear')
#Area Tiantan
dfwanliu['PM2.5'] = dfwanliu['PM2.5'].interpolate(method='linear')
dfwanliu['PM10'] = dfwanliu['PM10'].interpolate(method='linear')
#Area Wanliu
dfwanshouxigong['PM2.5'] = dfwanshouxigong['PM2.5'].interpolate(method='linear')
dfwanshouxigong['PM10'] = dfwanshouxigong['PM10'].interpolate(method='linear')
#Area Wanshouxigong

In [24]:
#pemeriksaan nilai interpolasi untuk PM
#dfaotizhongxin[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)
#dfchangping[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Changping)
#dfdingling[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dingling)
#dfdongsi[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dongsi)
#pemeriksaan hasil interpolasi untuk nilai dua jenis meter
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi PM (Aotizhongxing)
#dfchangping.isnull().sum() #untuk memeriksa kolom setelah interpolasi PM (Changping)

##Bagian 2 pembersihan dataset untuk pemeriksaan empat senyawa (dalam mikro meter/ meter kubik)

In [12]:
#pembersihan data SO2, NO2, CO O3 di dataset (Aotizhongxin sampai Wanshouxigong)
dfaotizhongxin[['SO2', 'NO2', 'CO', 'O3']] = dfaotizhongxin[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Aotizhongxin
dfchangping[['SO2', 'NO2', 'CO', 'O3']] = dfchangping[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Changping
dfdingling[['SO2', 'NO2', 'CO', 'O3']] = dfdingling[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Dingling
#dfdongsi[['SO2', 'NO2', 'CO', 'O3']] = dfdongsi[[['SO2', 'NO2', 'CO', 'O3']]].interpolate(method='linear') #Kota Dongsi
dfguanyuan[['SO2', 'NO2', 'CO', 'O3']] = dfguanyuan[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Guanyuan
dfgucheng[['SO2', 'NO2', 'CO', 'O3']] = dfgucheng[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Gucheng
dfhuairou[['SO2', 'NO2', 'CO', 'O3']] = dfhuairou[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Huairou
dfnonzhanguan[['SO2', 'NO2', 'CO', 'O3']] = dfnonzhanguan[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi[['SO2', 'NO2', 'CO', 'O3']] = dfshunyi[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Shunyi
dfwanliu[['SO2', 'NO2', 'CO', 'O3']] = dfwanliu[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong[['SO2', 'NO2', 'CO', 'O3']] = dfwanshouxigong[['SO2', 'NO2', 'CO', 'O3']].interpolate(method='linear') #Kota Wanshouxigong

#pemeriksaan interpolasi konsentrasi gas
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)

###Interpolasi suhu, tekanan, dan nilai tekanan dew secara linear

In [13]:
#persiapan proses interpolasi
dfaotizhongxin[['TEMP','PRES','DEWP']] = dfaotizhongxin[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Aotizhongxin
dfchangping[['TEMP','PRES','DEWP']] = dfchangping[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Changping
dfdingling[['TEMP','PRES','DEWP']] = dfdingling[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Dingling
dfdongsi[['TEMP','PRES','DEWP']] = dfdongsi[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Dongsi
dfguanyuan[['TEMP','PRES','DEWP']] = dfguanyuan[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Guanyuan
dfgucheng[['TEMP','PRES','DEWP']] = dfgucheng[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Gucheng
dfhuairou[['TEMP','PRES','DEWP']] = dfhuairou[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Huairou
dfnonzhanguan[['TEMP','PRES','DEWP']] = dfnonzhanguan[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi[['TEMP','PRES','DEWP']] = dfshunyi[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Shunyi
dfwanliu[['TEMP','PRES','DEWP']] = dfwanliu[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong[['TEMP','PRES','DEWP']] = dfwanshouxigong[['TEMP','PRES','DEWP']].interpolate(method='linear') #Kota Wanshouxigong


#Proses interpolasi untuk kadar hujan
dfaotizhongxin['RAIN'] = dfaotizhongxin['RAIN'].interpolate(method='linear') #Kota Aotizhongxin
dfchangping['RAIN'] = dfchangping['RAIN'].interpolate(method='linear') #Kota Changping
dfdingling['RAIN'] = dfdingling['RAIN'].interpolate(method='linear') #Kota Dingling
dfdongsi['RAIN'] = dfdongsi['RAIN'].interpolate(method='linear') #Kota Dongsi
dfguanyuan['RAIN'] = dfguanyuan['RAIN'].interpolate(method='linear') #Kota Guanyuan
dfgucheng['RAIN'] = dfgucheng['RAIN'].interpolate(method='linear') #Kota Gucheng
dfhuairou['RAIN'] = dfhuairou['RAIN'].interpolate(method='linear') #Kota Huairou
dfnonzhanguan['RAIN'] = dfnonzhanguan['RAIN'].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi['RAIN'] = dfshunyi['RAIN'].interpolate(method='linear') #Kota Shunyi
dfwanliu['RAIN'] = dfwanliu['RAIN'].interpolate(method='linear') #Kota Wanliu
dfwanshouxigong['RAIN'] = dfwanshouxigong['RAIN'].interpolate(method='linear') #Kota Wanshouxigong

#pemeriksaan hasil interpolasi
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi kadar suhu(Aotizhongxing)

#interpolasi tahap 2 untuk meningkatkan akurasi
untuk dua ukuran partikulat
1.   Ukuran PM2.5 (i)
2.   Ukuran PM10 (ii)



In [14]:
#persiapan interpolasi kriging untuk pengukuran nomor dan PM2.5 (untuk kedua belas kota)
mao = dfaotizhongxin['No'].values
pmai = dfaotizhongxin['PM2.5'].values #Kota Aotizhongxin
mch = dfchangping['No'].values
pmci = dfchangping['PM2.5'].values #Kota Changping
dli = dfdingling['No'].values
pmdi = dfdingling['PM2.5'].values #Kota Dingling
mdo = dfdongsi['No'].values
pmdoi = dfdongsi['PM2.5'].values #Kota Dongsi
mgua = dfguanyuan['No'].values
pmguai = dfguanyuan['PM2.5'].values #Kota Guanyuan
mguc = dfgucheng['No'].values
pmguci = dfgucheng['PM2.5'].values #Kota Gucheng
mhua = dfhuairou['No'].values
pmhuai = dfhuairou['PM2.5'].values #Kota Huairou
mnon = dfnonzhanguan['No'].values
pmnoni = dfnonzhanguan['PM2.5'].values #Kota Nonzhanguan
msh = dfshunyi['No'].values
pmshi = dfshunyi['PM2.5'].values #Kota Shunyi
mtia = dftiantan['No'].values
pmtiai = dftiantan['PM2.5'].values #Kota Tiantan
mwan = dfwanliu['No'].values
pmwani = dfwanliu['PM2.5'].values #Kota Wanliu
mwax = dfwanshouxigong['No'].values
pmwaxi = dfwanshouxigong['PM2.5'].values #Kota Wanshouxigong

#untuk ukuran PM10 lanjut ke bagian lain

In [15]:
#pengukuran PM10
mao = dfaotizhongxin['No'].values
pmaii = dfaotizhongxin['PM10'].values #Kota Aotizhongxin
mch = dfchangping['No'].values
pmcii = dfchangping['PM10'].values #Kota Changping
mdi = dfdingling['No'].values
pmdii = dfdingling['PM10'].values #Kota Dingling
mdo = dfdongsi['No'].values
pmdoii = dfdongsi['PM10'].values #Kota Dongsi
mgua = dfguanyuan['No'].values
pmguaii = dfguanyuan['PM10'].values #Kota Guanyuan
mguc = dfgucheng['No'].values
pmgucii = dfgucheng['PM10'].values #Kota Gucheng
mhua = dfhuairou['No'].values
pmhuaii = dfhuairou['PM10'].values #Kota Huairou

#Lanjut ke bagian selanjutnya

In [ ]:
#Dari Huairou
mnon = dfnonzhanguan['No'].values
pmnonii = dfnonzhanguan['PM10'].values #Kota Nonzhanguan
msh = dfshunyi['No'].values
pmshii = dfshunyi['PM10'].values #Kota Shunyi
mtia = dftiantan['No'].values
pmtiaii = dftiantan['PM10'].values #Kota Tiantan
mwan = dfwanliu['No'].values
pmwanii = dfwanliu['PM10'].values #Kota Wanliu
mwax = dfwanshouxigong['No'].values
pmwaxii = dfwanshouxigong['PM10'].values #Kota Wanshouxigong

#lanjut ke halaman lain untuk eksekusi

In [16]:
#persiapan model eksponensial untuk PM2.5 dan PM10
okao = okrig(x=mao, y=pmai, z=pmaii, variogram_model='exponential') #Aotizhongxin
okch = okrig(x=mch, y=pmci, z=pmcii, variogram_model='exponential') #Changping
okdli = okrig(x=dli, y=pmdi, z=pmdii, variogram_model='exponential') #Dingling
okdo = okrig(x=mdo, y=pmdoi, z=pmdoii, variogram_model='exponential') #Dongsi
okgua = okrig(x=mgua, y=pmguai, z=pmguaii, variogram_model='exponential') #Guanyuan
okguc = okrig(x=mguc, y=pmguci, z=pmgucii, variogram_model='exponential') #Gucheng
okhua = okrig(x=mhua, y=pmhuai, z=pmhuaii, variogram_model='exponential') #Huairou
oknon = okrig(x=mnon, y=pmnoni, z=pmnonii, variogram_model='exponential') #Nonzhanguan
oksh = okrig(x=msh, y=pmshi, z=pmshii, variogram_model='exponential') #Shunyi
oktia = okrig(x=mtia, y=pmtiai, z=pmtiaii, variogram_model='exponential') #Tiantan
okwal = okrig(x=mwan, y=pmwani, z=pmwanii, variogram_model='exponential') #Wanliu
okwax = okrig(x=mwax, y=pmwaxi, z=pmwaxii, variogram_model='exponential') #Wanshouxigong
#lanjut ke prediksi

In [18]:
#perincian nilai 'grid' untuk interpolasi masing-masing kota (pm2.5 adalah sumbu x)
grixao = np.arange(start=dfaotizhongxin['PM2.5'].min(), stop=dfaotizhongxin['PM2.5'].max(), step=1, dtype=int)
grixch = np.arange(start=dfchangping['PM2.5'].min(), stop=dfchangping['PM2.5'].max(), step=1, dtype=int)
grixdl = np.arange(start=dfdingling['PM2.5'].min(), stop=dfdingling['PM2.5'].max(), step=1, dtype=int)
grixdo = np.arange(start=dfdongsi['PM2.5'].min(), stop=dfdongsi['PM2.5'].max(), step=1, dtype=int)
grixgua = np.arange(start=dfguanyuan['PM2.5'].min(), stop=dfguanyuan['PM2.5'].max(), step=1, dtype=int)
grixguc = np.arange(start=dfgucheng['PM2.5'].min(), stop=dfgucheng['PM2.5'].max(), step=1, dtype=int)
grixhua = np.arange(start=dfhuairou['PM2.5'].min(), stop=dfhuairou['PM2.5'].max(), step=1, dtype=int)
grixnon = np.arange(start=dfnonzhanguan['PM2.5'].min(), stop=dfnonzhanguan['PM2.5'].max(), step=1, dtype=int)
grixsh = np.arange(start=dfshunyi['PM2.5'].min(), stop=dfshunyi['PM2.5'].max(), step=1, dtype=int)
grixtia = np.arange(start=dftiantan['PM2.5'].min(), stop=dftiantan['PM2.5'].max(), step=1, dtype=int)
grixwan = np.arange(start=dfwanliu['PM2.5'].min(), stop=dfwanliu['PM2.5'].max(), step=1, dtype=int)
grixwax = np.arange(start=dfwanshouxigong['PM2.5'].min(), stop=dfwanshouxigong['PM2.5'].max(), step=1, dtype=int)
#Bagian PM10 di bagian selanjutnya

In [19]:
#Bagian PM10 (sumbu y)
griyao = np.arange(start=dfaotizhongxin['PM10'].min(), stop=dfaotizhongxin['PM10'].max(), step=1, dtype=int)
griych = np.arange(start=dfchangping['PM10'].min(), stop=dfchangping['PM10'].max(), step=1, dtype=int)
griydl = np.arange(start=dfdingling['PM10'].min(), stop=dfdingling['PM10'].max(), step=1, dtype=int)
griydo = np.arange(start=dfdongsi['PM10'].min(), stop=dfdongsi['PM10'].max(), step=1, dtype=int)
griygua = np.arange(start=dfguanyuan['PM10'].min(), stop=dfguanyuan['PM10'].max(), step=1, dtype=int)
griyguc = np.arange(start=dfgucheng['PM10'].min(), stop=dfgucheng['PM10'].max(), step=1, dtype=int)
griyhua = np.arange(start=dfhuairou['PM10'].min(), stop=dfhuairou['PM10'].max(), step=1, dtype=int)
griynon = np.arange(start=dfnonzhanguan['PM10'].min(), stop=dfnonzhanguan['PM10'].max(), step=1, dtype=int)
griysh = np.arange(start=dfshunyi['PM10'].min(), stop=dfshunyi['PM10'].max(), step=1, dtype=int)
griytia = np.arange(start=dftiantan['PM10'].min(), stop=dftiantan['PM10'].max(), step=1, dtype=int)
griywan = np.arange(start=dfwanliu['PM10'].min(), stop=dfwanliu['PM10'].max(), step=1, dtype=int)
griywax = np.arange(start=dfwanshouxigong['PM10'].min(), stop=dfwanshouxigong['PM10'].max(), step=1, dtype=int)


In [1]:
#eksekusi nilai 'grid' untuk enam kota kota
zao = okao.execute(mao, grixao, griyao) #Aotizhongxin
zch = okch.execute(mch,grixch, griych) #Changpin
zdl = okdli.execute(mdi, grixdl, griydl) #Dingling
zdo = okdo.execute(mdo, grixdo, griydo) #Dongsi
zgua = okgua.execute(mgua, grixgua, griygua) #Guanyuan
zgu =  okguc.execute(mguc, grixguc, griyguc) #Gucheng
zhua = okhua.execute(mhua, grixhua, griyhua) #Huairou

NameError: name 'okao' is not defined

In [ ]:
#dari non zhanguan
znon = oknon.execute(mnon, grixnon, griynon) #Nonzhanguan
zsh = oksh.execute(msh, grixsh, griysh) #Shunyi
ztia = oktia.execute(mtia, grixtia, griytia) #Tiantan
zwan = okwal.execute(mwan, grixwan, griywan) #Wanliu
zwax = okwax.execute(mwax, grixwax, griywax) #Wanshouxigong

##Untuk empat gas (dalam mikrometer/m^3)
Empat gas ini memiliki penandaan yang berbeda
*   Kode untuk gas SO2 adalah 's'
*   Kode untuk gas NO adalah 'n'
* Kode untuk gas CO adalah 'c'
* Kode untuk gas O3 adalah 'o'



In [ ]:
#gas NO dan gas SO2 ('s' dan 'n')
mao = dfaotizhongxin['No'].values
so2ai = dfaotizhongxin['SO2'].values #Kota Aotizhongxin
no2ai = dfaotizhongxin['NO2'].values #Kota Aotizhongxin
mch = dfchangping['No'].values
so2ci = dfchangping['SO2'].values #Kota Changping
no2ci = dfchangping['NO2'].values #Kota Changping
dli = dfdingling['No'].values
so2di = dfdingling['SO2'].values #Kota Dingling
no2di = dfdingling['NO2'].values #Kota Dingling
mdo = dfdongsi['No'].values
so2doi = dfdongsi['SO2'].values #Kota Dongsi
no2doi = dfdongsi['NO2'].values #Kota Dongsi


In [ ]:
#dari guanyuan ke kota Shunyi
mgua = dfguanyuan['No'].values
so2gua = dfguanyuan['SO2'].values #Kota Guanyuan
no2gua = dfguanyuan['NO2'].values #Kota Guanyuan
mguc = dfgucheng['No'].values
so2guci = dfgucheng['SO2'].values #Kota Gucheng
no2guci = dfgucheng['NO2'].values #Kota Gucheng
mhua = dfhuairou['No'].values
so2huai = dfhuairou['SO2'].values #Kota Huairou
no2huai = dfhuairou['NO2'].values #Kota Huairou
mnon = dfnonzhanguan['No'].values
so2noni = dfnonzhanguan['SO2'].values #Kota Nonzhanguan
no2noni = dfnonzhanguan['NO2'].values #Kota Nonzhanguan
mosh = dfshunyi['No'].values
so2shi = dfshunyi['SO2'].values #Kota Shunyi
no2shi = dfshunyi['NO2'].values #Kota Shunyi



In [ ]:
#Bagian akhir
mtia = dftiantan['No'].values
so2ti = dftiantan['SO2'].values #Kota Tiantan
no2ti = dftiantan['NO2'].values #Kota Tiantan
mowal = dfwanliu['No'].values
so2wani = dfwanliu['SO2'].values #Kota Wanliu
no2wani = dfwanliu['NO2'].values #Kota Wanliu
mowax = dfwanshouxigong['No'].values
so2waxi = dfwanshouxigong['SO2'].values #Kota Wanshouxigong
no2waxi = dfwanshouxigong['NO2'].values #Kota Wanshouxigong


#persiapan model untuk gas NO2 dan SO2
okaoii = okrig(x=mao, y=so2ai, z=no2ai, variogram_model='exponential') #Aotizhongxin
okchii = okrig(x=mch, y=so2ci, z=no2ci, variogram_model='exponential') #Changping
okdlii = okrig(x=dli, y=so2di, z=no2di, variogram_model='exponential') #Dingling
okdoii = okrig(x=mdo, y=so2doi, z=no2doi, variogram_model='exponential') #Dongsi
okguaii = okrig(x=mgua, y=so2gua, z=no2gua, variogram_model='exponential') #Guanyuan
okgucii = okrig(x=mguc, y=so2guci, z=no2guci, variogram_model='exponential') #Gucheng
okhuaii = okrig(x=mhua, y=so2huai, z=no2huai, variogram_model='exponential') #Huairou

In [ ]:
#lanjutan pemeriksaan (eksekusi untuk gas SO2 dan NO2)
oknonii = okrig(x=mnon, y=so2noni, z=no2noni, variogram_model='exponential')#Nonzhanguan
okshii = okrig(x=mosh, y=so2shi, z=no2shi, variogram_model='exponential') #Shunyi
oktia = okrig(x=mtia, y=so2ti, z=no2ti, variogram_model='exponential') #Tiantan

#tahap eksekusi awal

In [ ]:
#gas CO dan O3 ('c' dan 'o')
mao = dfaotizhongxin['No'].values
coai = dfaotizhongxin['CO'].values #Kota Aotizhongxin
ozai = dfaotizhongxin['O3'].values #Kota Aotizhongxin
mch = dfchangping['No'].values
cochi = dfchangping['CO'].values #Kota Changping
ozchi = dfchangping['O3'].values #Kota Changping
dli = dfdingling['No'].values
codi = dfdingling['CO'].values #Kota Dingling
ozdli = dfdingling['O3'].values #Kota Dingling
mdo = dfdongsi['No'].values
codoi = dfdongsi['CO'].values #Kota Dongsi
ozdoi = dfdongsi['O3'].values #Kota Dongsi
mgua = dfguanyuan['No'].values
cogua = dfguanyuan['CO'].values #Kota Guanyuan
ozgua = dfguanyuan['O3'].values #Kota Guanyuan
mguc = dfgucheng['No'].values
coguc = dfgucheng['CO'].values #Kota Gucheng
ozguc = dfgucheng['O3'].values #Kota Gucheng
mhua = dfhuairou['No'].values
cohuai = dfhuairou['CO'].values #Kota Huairou
ozhuai = dfhuairou['O3'].values #Kota Huairou



In [ ]:
#Lanjutan analisis (persiapan untuk gas CO dan O3)
okaoiiii = okrig(x=mao, y=coai, z=ozai, variogram_model='exponential') #Aotizhongxin
okchiii = okrig(x=mch, y=cochi, z=ozchi, variogram_model='exponential') #Changping

In [ ]:
#Bagian 2


##Pengisian data hilang untuk arah angin dan kecepatan


In [17]:
#persiapan pengisian data hilang di arah angin
dfaotizhongxin['wd'] = dfaotizhongxin['wd'].fillna('unknown', inplace=True) #Kota Aotizhongxin
dfchangping['wd'] = dfchangping['wd'].fillna('unknown', inplace=True) #Kota Changping
dfdingling['wd'] = dfdingling['wd'].fillna('unknown', inplace=True) #Kota Dingling
dfdongsi['wd'] = dfdongsi['wd'].fillna('unknown', inplace=True) #Kota Dongsi
dfguanyuan['wd'] = dfguanyuan['wd'].fillna('unknown', inplace=True) #Kota Guanyuan
dfgucheng['wd'] = dfgucheng['wd'].fillna('unknown', inplace=True) #Kota Gucheng
dfhuairou['wd'] = dfhuairou['wd'].fillna('unknown', inplace=True) #Kota Huairou
dfnonzhanguan['wd'] = dfnonzhanguan['wd'].fillna('unknown', inplace=True) #Kota Nonzhanguan
dfshunyi['wd'] = dfshunyi['wd'].fillna('unknown', inplace=True) #Kota Shunyi

#interpolasi linear untuk kecepatan angin
dfaotizhongxin['WSPM'] = dfaotizhongxin['WSPM'].interpolate(method='linear') #Kota Aotizhongxin
dfchangping['WSPM'] = dfchangping['WSPM'].interpolate(method='linear') #Kota Changping
dfdingling['WSPM'] = dfdingling['WSPM'].interpolate(method='linear') #Kota Dingling
dfdongsi['WSPM'] = dfdongsi['WSPM'].interpolate(method='linear') #Kota Dongsi
dfguanyuan['WSPM'] = dfguanyuan['WSPM'].interpolate(method='linear') #Kota Guanyuan
dfhuairou['WSPM'] = dfhuairou['WSPM'].interpolate(method='linear') #Kota Huairou
dfnonzhanguan['WSPM'] = dfnonzhanguan['WSPM'].interpolate(method='linear') #Kota Nonzhanguan
dfshunyi['WSPM'] = dfshunyi['WSPM'].interpolate(method='linear') #Kota Shunyi



## Exploratory Data Analysis (EDA)

**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

### Explore The insight

In [ ]:
# eksplirasi


**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
#Tahap awal visualisasi untuk pertanyaan 1


### Pertanyaan 2:

In [ ]:
#Tahap awal visualisasi untuk pertanyaan 1


**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2